## Installing Dependencies

In [ ]:
#!pip install langchain-openai unstructured==0.7.12 pinecone-client openai tiktoken
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


## Importing Dependencies

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.vectorstores.pinecone import Pinecone
from langchain.chains import RetrievalQA
from langchain import OpenAI
from langchain_openai import ChatOpenAI
import os
import nltk
import pinecone
import openai

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## User Input for URLs to build copilot on

In [ ]:
urls = []
n_weblinks = int(input("How many web links you want the copilot to refer for response generation & insights? Enter here: "))
print("Enter your links below: ")
for i in range(0, n_weblinks):
  inp = input()
  # appending the element in list
  urls.append(inp)

How many web links you want the copilot to refer for response generation & insights? Enter here: 1
Enter your links below: 
https://arxiv.org/pdf/2312.10997.pdf


## Loading URL through LangChain's UnstructuredURLLoader

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(urls=urls)
urls = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# Merging urls into a single list
documents = []
documents.extend(urls)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
texts = text_splitter.split_documents(documents)
texts

## OpenAI API Key Setting



In [ ]:
# Set the OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = "sk-xxxx"

## Pinecone API Setting

In [ ]:
#Set the pinecone key
pinecone.init(api_key="71ed08b1-xxxx", environment="gcp-starter")

In [ ]:
#Creating a Pinecone index automatically if index doesn't exists
embeddings = OpenAIEmbeddings()
index_name = 'myindex' #index name
if index_name not in pinecone.list_indexes():
    # Creating a new index
    pinecone.create_index(name=index_name, metric="cosine", dimension=1536)
# The OpenAI embedding model 'text-embedding-ada-002 uses 1536 dimensions'
docsearch = Pinecone.from_documents(texts, embeddings, index_name=index_name)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## Model Building and Execution

In [ ]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4", max_tokens=256)
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=docsearch.as_retriever(),
        verbose=True,
        return_source_documents=True)

In [ ]:
initial_prompt = "You are a researcher who is going to search the web links, summarize them and share insights as asked"

In [ ]:
query = "What is a RAG framework?"
result = qa({"query": query, "prompt": initial_prompt})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [ ]:
print(result['result'])

The RAG framework refers to a research paradigm in the field of technology. It has evolved over time and is categorized into three types: Naive RAG, Advanced RAG, and Modular RAG. These categories were developed to address specific shortcomings in the initial RAG model. The framework involves various technologies that work together to form a cohesive and effective system. It is used for information retrieval and context-aware generation. The performance of RAG models is assessed using certain metrics and benchmarks.


In [ ]:
result['source_documents']

[Document(page_content='3 RAG Framework The RAG research paradigm is continuously evolving, and this section primarily delineates its progression. We cate- gorize it into three types: Naive RAG, Advanced RAG, and Modular RAG. While RAG were cost-effective and surpassed the performance of the native LLM, they also exhibited sev- eral limitations. The development of Advanced RAG and Modular RAG was a response to these specific shortcomings in Naive RAG.', metadata={'source': 'https://arxiv.org/pdf/2312.10997.pdf'}),
 Document(page_content='technologies embedded in each of these critical components, providing a profound understanding of the advancements in RAG systems. Furthermore, this paper introduces the metrics and benchmarks for assessing RAG models, along with the most up-to-date evaluation In conclusion, the paper delineates framework. including the prospective avenues for research, identification of challenges, the expansion of multi-modalities, and the progression of the RAG infr